In [1]:
import numpy as np
import math
import random

Załadujemy zbiór danych - obrazków cyfr z pakietu sklearn:

In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape)
(1797, 64)


(1797, 64)


(1797, 64)

Wyświetlenie pierwszego z obrazków w kolekcji (indeks 0). Możesz spróbować wyświetlić kilka kolejnych.

In [10]:
import matplotlib.pyplot as plt 
plt.gray() 
plt.matshow(digits.images[13]) 
plt.show() 

Tak powinien wyglądać label (etykieta) tego obrazka:

In [9]:
digits.target[27]

7

Sprawdzenie wielkosci zbioru danych:

In [11]:
print(digits['target'].shape)

(1797,)


In [12]:
print(digits['images'].shape)

(1797, 8, 8)


Praca z obrazkami wymaga spłaszczenia ich do 1-wymiarowego wektora. Podpowiedź: użyj komendy numpy.reshape!

In [13]:
X = digits['images'].reshape(8*8, 1797)

In [14]:
Y = digits['target']

In [15]:
X.shape

(64, 1797)

In [16]:
Y.shape

(1797,)

In [17]:
np.unique(Y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Regresja logistyczna pozwala na binarną klasyfikację (tylko dwie klasy), spróbujmy więc nauczyć nasz model rozpoznawania zer i jedynek. Wyciągnijmy więc tylko te dwa zbiory liczb w podziale na X i Y:

In [18]:
ix = np.isin(Y, [0,1])
ix = np.in1d(Y, [0,1])
X_zeroone = np.compress(ix, X, axis=1)
Y_zeroone = np.compress(ix, Y, axis=0)

Na tym etapie, X_zeroone i Y_zeroone zawierają wyłącznie obrazki oraz etykiety dla zer i jedynek.

In [19]:
X_zeroone.shape

(64, 360)

In [20]:
np.unique(Y_zeroone)

array([0, 1])

#### Zadanie
Podziel X_zeroone i Y_zeroone na zbiory treningowe _train oraz testowe _test. Niech pierwsze 300 przypadków należy do zbioru treningowego:

In [27]:
takefirst=300
X_train, X_test = X_zeroone[:,:takefirst], X_zeroone[:,takefirst:]
Y_train, Y_test = Y_zeroone[:takefirst], Y_zeroone[takefirst:]

#### Zadanie
Użyj funkcji np_sigmoid z arkusza zadań z numpym

In [21]:
def np_sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s


In [22]:
def inicjalizuj_zerami(dim):
    w = np.zeros((dim,1))
    b = 0
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    return w, b    

Zaimplementuj metodę propaguj, a w niej:
<img src="images/wzory_regresja_logistyczna.png">
W tym przypadku * oznacza iloczyn macierzowy, .T transpozycję macierzy.
Przy obliczaniu aktywacji czyli A, funkcją zewnętrzną jest sigmoid.

In [32]:
def propaguj(w, b, X, Y):
    m = X.shape[1]
    # propagacja w przod, z X liczymy koszt:
    A = np_sigmoid(np.dot(w.T,X)+b)                                  
    cost = - np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))/takefirst        
    
    # propagacja wsteczna, czyli liczenie gradientu:
    dw = np.dot(X,(A-Y).T)/m
    db = np.sum(A-Y)/m
    
    cost = np.squeeze(cost)
    
    grads = {"dw": dw, "db": db}
    return grads, cost

Zaimplementuj aktualizację obu parametrów (wag oraz biasów) modelu zgodnie z równaniem:

$$p = p - \lambda * dp$$

Gdzie $\lambda$ to learning rate a $dp$ to gradient parametru p.

In [33]:
def optymalizuj(w, b, X, Y, num_iterations, learning_rate):
    costs = []
    m = X.shape[1]
    for i in range(num_iterations):
        grads, cost = propaguj(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate*dw
        b = b - learning_rate*db
        costs.append(cost)
    
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    
    return params, grads, costs        

Zaimplementuj metodę klasyfikuj, a w niej:
 - funkcję aktywacji A według równania na A powyżej
 - oblicz wartości przewidywanych etykiet T_prediction jako: 0 jeśli [A,i] jest mniejszy lub równy niż 0.5, 1 w przeciwnym przypadku.

In [45]:
def klasyfikuj(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = np_sigmoid( np.dot(w.T,X) +b)
    for i in range(A.shape[1]):
        if A[0,i] <= 0.5: Y_prediction[0,i] = 0
        else: Y_prediction[0,i] = 1
    return Y_prediction

In [46]:
m = X.shape[1]
w, b = inicjalizuj_zerami(X.shape[0])
num_iterations=500
learning_rate=0.1
params, grads, costs = optymalizuj(w, b, X_train, Y_train, num_iterations, learning_rate)

Y_prediction_test = klasyfikuj(w, b, X_test)
Y_prediction_train = klasyfikuj(w, b, X_train)
print("------>  learning rate:"+str(learning_rate))
print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    

------>  learning rate:0.1
train accuracy: 50.0 %
test accuracy: 46.6666666667 %


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in multiply


### Zadania
1. Przetestuj różne zakresy parametru liczba iteracji (num_iterations), np 10, 100, 500
2. Dla każdej wartości num_iterations, przetestuj learning_rate w zakresie np   1.5, 1.0, 0.01, 00.1
3. Spróbuj uzależnić learning_rate od num_iterations, tak żeby learning_rate malał co każde N iteracji. Dobierz N do liczby iteracji.

In [70]:
w, b = inicjalizuj_zerami(X.shape[0])
m = X.shape[1]

for num_iterations in [10,100,500]:
    for learning_rate in [1.5, 1.0, 0.01, 0.01]:
        params, grads, costs = optymalizuj(w, b, X_train, Y_train, num_iterations, learning_rate)
        w = params["w"]
        b = params["b"]

        Y_prediction_test = klasyfikuj(w, b, X_test)
        Y_prediction_train = None
        print("------>  learning rate:"+str(learning_rate))
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in multiply


------>  learning rate:0.8
train accuracy: 54.3333333333 %
test accuracy: 56.6666666667 %
------>  learning rate:0.1
train accuracy: 61.3333333333 %
test accuracy: 70.0 %
------>  learning rate:0.01
train accuracy: 61.0 %
test accuracy: 68.3333333333 %
------>  learning rate:0.001
train accuracy: 61.3333333333 %
test accuracy: 68.3333333333 %
------>  learning rate:0.0001
train accuracy: 61.3333333333 %
test accuracy: 68.3333333333 %


Dla jakich ustawień udało się uzyskać najwyższą dokładność (accuracy) na zbiorze testowym?
